In [ ]:
# %% [markdown]
# # 🤖 WhatsApp Chat Style Mimicker
# 
# Este script analiza un chat de WhatsApp y puede imitar cómo habla cada persona.
# 
# ## 📝 Antes de empezar:
# 1. Exporta un chat de WhatsApp (sin medios)
# 2. Pon el archivo .txt en esta carpeta
# 3. Crea un archivo .env con tu API key de OpenAI

# %% [markdown]
# ## 📚 Importar lo necesario

# %%
import os
import re
import pandas as pd
from datetime import datetime
from openai import OpenAI
from dotenv import load_dotenv

# Cargar API key
load_dotenv()

# %% [markdown]
# ## 📂 Buscar archivos de chat

# %%
# Ver qué archivos de chat hay disponibles
chat_files = [f for f in os.listdir('chats') if f.endswith('.txt')]

print("📁 Archivos encontrados:")
for i, file in enumerate(chat_files, 1):
    print(f"{i}. {file}")

if not chat_files:
    print("❌ No se encontraron archivos .txt")
    raise Exception("Necesitas exportar un chat de WhatsApp primero")


In [ ]:
import pandas as pd
import re
import os  # Agregué esta importación que faltaba

# Elegir un archivo
chat_files = [f for f in os.listdir('chats') if f.endswith('.txt')]  # Asumiendo que tienes una lista de archivos
file_number = int(input("\nElige el número del archivo a analizar: "))
selected_file = chat_files[file_number - 1]
print(f"Archivo seleccionado: {selected_file}")

# %% [markdown]
# ## 📖 Leer y procesar el chat

# Leer el archivo y mostrar las primeras líneas para debug
with open(os.path.join('chats', selected_file), 'r', encoding='utf-8') as file:  # Corregí la ruta del archivo
    content = file.read()

print("Primeras 5 líneas del archivo:")
print("\n".join(content.split('\n')[:5]))
print("\nTotal de líneas:", len(content.split('\n')))

# Intentar diferentes patrones de regex para WhatsApp
patterns = {
    'android': [
        # Formato Android: "dd/mm/yy, HH:MM - Nombre: Mensaje"
        r'(\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2})\s-\s([^:]+):\s(.+)',
        # Formato Android AM/PM: "dd/mm/yy, HH:MM AM/PM - Nombre: Mensaje"
        r'(\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s[APMapm]{2})\s-\s([^:]+):\s(.+)'
    ],
    'iphone': [
        # Formato iPhone con segundos: "[dd/mm/yy, HH:MM:SS] Nombre: Mensaje"
        r'\[(\d{1,2}/\d{1,2}/\d{2},\s\d{1,2}:\d{2}:\d{2})\]\s([^:]+):\s(.+)',
        # Formato iPhone sin segundos: "[dd/mm/yy, HH:MM] Nombre: Mensaje"
        r'\[(\d{1,2}/\d{1,2}/\d{2},\s\d{1,2}:\d{2})\]\s([^:]+):\s(.+)'
    ]
}

matches = []
used_pattern = None

# Probar cada grupo de patrones
for platform, pattern_list in patterns.items():
    for pattern in pattern_list:
        matches = re.findall(pattern, content, re.MULTILINE)
        if matches:
            used_pattern = pattern
            print(f"\n✅ Patrón encontrado: {pattern}")
            print(f"Mensajes encontrados: {len(matches)}")
            break
    if matches:
        break

if not matches:
    print("\n❌ No se pudo encontrar un patrón válido en el archivo")
    print("Por favor, verifica que el archivo es una exportación de WhatsApp")
    raise Exception("Formato de archivo no reconocido")

# Depuración: Mostrar las primeras coincidencias para verificar la estructura
print("\nPrimeras 5 coincidencias:")
for i, match in enumerate(matches[:5]):
    print(f"Coincidencia {i+1}: {match}")

# Verificar que cada coincidencia tiene 3 elementos
if not all(len(match) == 3 for match in matches):
    raise ValueError("Las coincidencias no tienen 3 elementos (datetime, sender, message). Revisa el patrón o el archivo.")

# Crear DataFrame
df = pd.DataFrame(matches, columns=['datetime', 'sender', 'message'])

# Mostrar estructura del DataFrame
print("\nEstructura del DataFrame:")
print(df.head())


In [ ]:

# Ver miembros del chat
members = df['sender'].unique().tolist()


print(f"\n👥 Miembros encontrados ({len(members)}):")
for i, member in enumerate(members, 1):
    print(f"{i}. {member}")

if not members:
    print("❌ No se encontraron miembros en el chat")
    raise Exception("No se pudieron extraer los miembros del chat")

# %% [markdown]
# ## 🎯 Seleccionar miembro y analizar su estilo


In [ ]:
import random

print("💬 Generando un mensaje típico de cada miembro...")

# Para cada miembro, generar un mensaje típico
for member in members:
    print(f"\n👤 Generando mensaje típico de: {member}")
    print("=" * 50)
    
    # Obtener mensajes del miembro
    member_messages = df[df['sender'] == member]['message'].tolist()
    
    # Seleccionar aleatoriamente 10 mensajes si hay más de 10, o todos si hay menos
    if len(member_messages) > 500:
        sample_messages = random.sample(member_messages, 500)
    else:
        sample_messages = member_messages
    
    # Crear el prompt del sistema
    system_prompt = f"""Estás imitando el estilo de habla de una persona de WhatsApp.
Aquí hay algunos mensajes de ejemplo de esta persona:

{chr(10).join(f'- {msg}' for msg in sample_messages)}

Genera una respuesta que coincida con el estilo de habla de esta persona, capturando tanto su forma de expresarse como los temas que suele abordar. La respuesta debe reflejar fielmente su tono, vocabulario, uso de emojis y patrones de escritura. Presta especial atención a los siguientes aspectos:

- **Tono y vocabulario**: Imita su nivel de formalidad, jerga o palabras clave que usa frecuentemente.
- **Uso de emojis**: Incluye emojis de la misma manera que la persona los usa, ya sea de forma frecuente, moderada o esporádica.
- **Patrones de escritura**: Refleja su estilo, como el uso de abreviaturas, signos de puntuación, mayúsculas o estructuras de frases particulares.
- **Temas recurrentes**: Incorpora temas o asuntos que la persona menciona habitualmente, asegurando que la respuesta parezca algo que podría decir en una conversación real.
- **Personalidad**: La respuesta debe capturar su actitud general, ya sea optimista, pesimista, sarcástica, directa, etc. Evita hacerla excesivamente positiva si la persona no lo es.

El objetivo es que la respuesta no solo suene como algo que la persona podría decir, sino que también represente su personalidad y los temas que le interesan. Usa el contexto proporcionado para guiar la respuesta, pero asegúrate de que se sienta auténtica y coherente con su estilo de comunicación. 
"""

    # Mensaje del usuario genérico
    user_message = "Genera una frase típica de esta persona basada en sus mensajes anteriores."

    try:
        # Generar respuesta usando OpenAI
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            temperature=0.7,
            max_tokens=4000
        )

        print(f"💭 Respuesta: {response.choices[0].message.content}")

    except Exception as e:
        print(f"❌ Error generando mensaje para {member}: {str(e)}")
    print("-" * 50)

In [ ]:
'''#chatbot
print("💬 Generando un mensaje típico de cada miembro...")

# Lista de prompts
prompts = [
    # Relacionados con bebidas/salir
    "Que te parece lo de Ucrania? ",
    
    
]

# Para cada prompt
for p in prompts:
    print(f"\n📌 Procesando prompt: {p}")
    print("=" * 50)
    
    # Para cada miembro, generar un mensaje típico
    for member in members:
        print(f"\n👤 Generando mensaje típico de: {member}")
        
        # Obtener mensajes del miembro
        member_messages = df[df['sender'] == member]['message'].tolist()
        sample_messages = member_messages[-10:] if len(member_messages) > 10 else member_messages
        
        # Crear el prompt del sistema
        system_prompt = f"""Estás imitando el estilo de habla de una persona de WhatsApp.
Aquí hay algunos mensajes de ejemplo de esta persona:

{chr(10).join(f'- {msg}' for msg in sample_messages)}

Genera una respuesta que coincida con el estilo de habla de esta persona, capturando tanto su forma de expresarse como los temas que suele abordar. La respuesta debe reflejar fielmente su tono, vocabulario, uso de emojis y patrones de escritura. Presta especial atención a los siguientes aspectos:

- **Tono y vocabulario**: Imita su nivel de formalidad, jerga o palabras clave que usa frecuentemente.
- **Uso de emojis**: Incluye emojis de la misma manera que la persona los usa, ya sea de forma frecuente, moderada o esporádica.
- **Patrones de escritura**: Refleja su estilo, como el uso de abreviaturas, signos de puntuación, mayúsculas o estructuras de frases particulares.
- **Temas recurrentes**: Incorpora temas o asuntos que la persona menciona habitualmente, asegurando que la respuesta parezca algo que podría decir en una conversación real.
- **Personalidad**: La respuesta debe capturar su actitud general, ya sea optimista, pesimista, sarcástica, directa, etc. Evita hacerla excesivamente positiva si la persona no lo es.

El objetivo es que la respuesta no solo suene como algo que la persona podría decir, sino que también represente su personalidad y los temas que le interesan. Usa el contexto proporcionado para guiar la respuesta, pero asegúrate de que se sienta auténtica y coherente con su estilo de comunicación.
No tienes que ser excesivamente positivo si la persona no lo es
"""

        try:
            # Generar respuesta usando OpenAI
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": p}
                ],
                temperature=0.7,
                max_tokens=100
            )

            print(f"💭 Respuesta: {response.choices[0].message.content}")

        except Exception as e:
            print(f"❌ Error generando mensaje para {member}: {str(e)}")
        print("-" * 50)'''